In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [2]:
df = spark.read.json('/home/suhasini/Our Data/Our Projects/Python/Python-Udemy/Spark&Python4BigDataWithPySpark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json')

In [3]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [4]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [5]:
df.describe()

DataFrame[summary: string, age: string, name: string]

In [6]:
#Define Schema manually-Force data to follow user defined schema rather letting Spark infer on its own.

from pyspark.sql.types import StructField, StructType, IntegerType, StringType
schema = [StructField('age', IntegerType(), True), 
          StructField('name', StringType(),True)]

final_schema = StructType(fields = schema)

df = spark.read.json('/home/suhasini/Our Data/Our Projects/Python/Python-Udemy/Spark&Python4BigDataWithPySpark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/people.json', schema = final_schema) 

In [7]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- name: string (nullable = true)



In [8]:
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [9]:
df1 =df.withColumn('newAge', df['age']-10).show()

+----+-------+------+
| age|   name|newAge|
+----+-------+------+
|null|Michael|  null|
|  30|   Andy|    20|
|  19| Justin|     9|
+----+-------+------+



In [11]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
df.createOrReplaceTempView('people')

In [14]:
result = spark.sql('SELECT * FROM people')
result.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [16]:
df = spark.read.csv('/home/suhasini/Our Data/Our Projects/Python/Python-Udemy/Spark&Python4BigDataWithPySpark/Python-and-Spark-for-Big-Data-master/Spark_DataFrames/appl_stock.csv', inferSchema=True, header=True)

In [17]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [18]:
df.show()

+----------+------------------+------------------+------------------+------------------+---------+------------------+
|      Date|              Open|              High|               Low|             Close|   Volume|         Adj Close|
+----------+------------------+------------------+------------------+------------------+---------+------------------+
|2010-01-04|        213.429998|        214.499996|212.38000099999996|        214.009998|123432400|         27.727039|
|2010-01-05|        214.599998|        215.589994|        213.249994|        214.379993|150476200|27.774976000000002|
|2010-01-06|        214.379993|            215.23|        210.750004|        210.969995|138040000|27.333178000000004|
|2010-01-07|            211.75|        212.000006|        209.050005|            210.58|119282800|          27.28265|
|2010-01-08|        210.299994|        212.000006|209.06000500000002|211.98000499999998|111902700|         27.464034|
|2010-01-11|212.79999700000002|        213.000002|      

In [20]:
df.count()

1762

In [24]:
df.filter('Close > 500').select(['open','close']).show()

+------------------+------------------+
|              open|             close|
+------------------+------------------+
|        499.529991|502.60002099999997|
|        504.659988|        509.459991|
|        491.500008|502.20999900000004|
|        503.109993|         502.12001|
|506.88001299999996|        514.850021|
|        513.079994|        513.039993|
|        515.079987| 516.3899769999999|
| 519.6699980000001| 522.4099809999999|
|        521.309982|        525.760017|
|        527.960014|        535.410011|
| 541.5600049999999|        542.440025|
|        548.169983| 544.4699780000001|
|        544.240013|        545.180008|
|        545.420013| 533.1600269999999|
|        523.659996|        530.259987|
| 536.8000030000001| 530.6900099999999|
| 534.6899950000001|        541.989975|
|        544.209999|        545.170021|
| 548.9799879999999|        551.999977|
|        557.540024|        568.099998|
+------------------+------------------+
only showing top 20 rows



In [23]:
df.filter('Close < 500').select(['open','close']).count()

1359

In [28]:
result = df.filter((df['Close'] > 500) & (df['open'] > 500)).select(['open','close']).collect()

In [42]:
from pyspark.sql.functions import format_number

df.filter((df['Close'] > 500) & (df['open'] > 500)).select(['open','close']).select(format_number('open',2).alias('OPEN'),format_number('close',2).alias('CLOSE')).show()

+------+------+
|  OPEN| CLOSE|
+------+------+
|504.66|509.46|
|503.11|502.12|
|506.88|514.85|
|513.08|513.04|
|515.08|516.39|
|519.67|522.41|
|521.31|525.76|
|527.96|535.41|
|541.56|542.44|
|548.17|544.47|
|544.24|545.18|
|545.42|533.16|
|523.66|530.26|
|536.80|530.69|
|534.69|541.99|
|544.21|545.17|
|548.98|552.00|
|557.54|568.10|
|578.05|589.58|
|599.61|585.56|
+------+------+
only showing top 20 rows



In [49]:
df.filter((df['open']<500) & ~(df['close']<500)).select(['open','close','volume']) \
                    .select(format_number('open',3).alias('OPEN AMT'), \
                            format_number('close',3).alias('CLOSE AMT'), \
                            format_number('volume',2).alias('VOLUME')).show()

+--------+---------+--------------+
|OPEN AMT|CLOSE AMT|        VOLUME|
+--------+---------+--------------+
| 499.530|  502.600|129,304,000.00|
| 491.500|  502.210|236,138,000.00|
| 494.640|  506.090|172,701,200.00|
| 498.520|  500.000|118,230,700.00|
| 499.980|  504.500| 63,398,300.00|
| 495.180|  500.600|116,199,300.00|
+--------+---------+--------------+



In [51]:
result = df.filter(df['Low'] == 197.16).collect()

In [54]:
result[0].asDict()

{'Date': '2010-01-22',
 'Open': 206.78000600000001,
 'High': 207.499996,
 'Low': 197.16,
 'Close': 197.75,
 'Volume': 220441900,
 'Adj Close': 25.620401}